In [1]:
from ctgan import CTGAN
from data_transformer import DataTransformer
from data_sampler import DataSampler
from train import Train
import numpy as np
import tensorflow as tf

2023-06-08 11:16:42.937926: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
import pandas as pd
df = pd.read_csv('insurance_claims.csv')
df

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,...,YES,71610,6510,13020,52080,Saab,92x,2004,Y,NaN
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,...,?,5070,780,780,3510,Mercedes,E400,2007,Y,NaN
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,...,NO,34650,7700,3850,23100,Dodge,RAM,2007,N,NaN
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,...,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y,NaN
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,...,NO,6500,1300,650,4550,Accura,RSX,2009,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,38,941851,1991-07-16,OH,500/1000,1000,1310.80,0,431289,...,?,87200,17440,8720,61040,Honda,Accord,2006,N,NaN
996,285,41,186934,2014-01-05,IL,100/300,1000,1436.79,0,608177,...,?,108480,18080,18080,72320,Volkswagen,Passat,2015,N,NaN
997,130,34,918516,2003-02-17,OH,250/500,500,1383.49,3000000,442797,...,YES,67500,7500,7500,52500,Suburu,Impreza,1996,N,NaN
998,458,62,533940,2011-11-18,IL,500/1000,2000,1356.92,5000000,441714,...,YES,46980,5220,5220,36540,Audi,A5,1998,N,NaN


In [4]:

raw_data = df[['injury_claim', 'property_claim', 'vehicle_claim', 'auto_make', 'auto_model', 'auto_year']]

transformer = DataTransformer()
transformer.fit(raw_data, discrete_columns=('auto_make', 'auto_model', 'auto_year'))
data_t = transformer.transform(raw_data)      #matrix of transformed data
output_info = transformer.output_info_list

log_frequency = True
sampler = DataSampler(data_t, output_info, log_frequency)


In [5]:
output_info

[[SpanInfo(dim=1, activation_fn='tanh'),
  SpanInfo(dim=3, activation_fn='softmax')],
 [SpanInfo(dim=1, activation_fn='tanh'),
  SpanInfo(dim=5, activation_fn='softmax')],
 [SpanInfo(dim=1, activation_fn='tanh'),
  SpanInfo(dim=5, activation_fn='softmax')],
 [SpanInfo(dim=14, activation_fn='softmax')],
 [SpanInfo(dim=39, activation_fn='softmax')],
 [SpanInfo(dim=21, activation_fn='softmax')]]

In [6]:
transformer._column_transform_info_list

[ColumnTransformInfo(column_name='injury_claim', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=3, activation_fn='softmax')], output_dimensions=4),
 ColumnTransformInfo(column_name='property_claim', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=5, activation_fn='softmax')], output_dimensions=6),
 ColumnTransformInfo(column_name='vehicle_claim', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=5, activation_fn='softmax')], output_dimensions=6),
 ColumnTransformInfo(column_name='auto_make', column_type='discrete', transform=OneHotEncoder(), output_info=[SpanInfo(dim=14, activation_fn='softmax')], output_dimensions=14),
 ColumnTransformInfo(column_name='auto_model', column_type='d

In [7]:
data_t.shape

(1000, 90)

In [9]:
model = CTGAN()
generator  = model.make_generator(sampler, transformer)
discriminator = model.make_discriminator(sampler, transformer)

In [10]:
generator.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 202)]             0         
_________________________________________________________________
residual_layer_2 (ResidualLa (None, 458)               51968     
_________________________________________________________________
residual_layer_3 (ResidualLa (None, 714)               117504    
_________________________________________________________________
dense_2 (Dense)              (None, 90)                64350     
Total params: 233,822
Trainable params: 233,822
Non-trainable params: 0
_________________________________________________________________


In [11]:
discriminator.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1640)]            0         
_________________________________________________________________
discriminator_layer_2 (Discr (None, 256)               420096    
_________________________________________________________________
discriminator_layer_3 (Discr (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 486,145
Trainable params: 486,145
Non-trainable params: 0
_________________________________________________________________


In [12]:
from train import Train
train = Train(transformer, sampler, generator, discriminator,  epochs=100)

In [13]:
import tensorflow as tf
train.train(raw_data)

2023-06-08 12:03:47.576511: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


Epoch 1, Loss G:  1.0700, Loss D:  7.7152
Epoch 2, Loss G:  1.0542, Loss D:  7.0842
Epoch 3, Loss G:  1.0422, Loss D:  6.2515
Epoch 4, Loss G:  1.0443, Loss D:  5.1835
Epoch 5, Loss G:  1.0071, Loss D:  4.0829
Epoch 6, Loss G:  1.0117, Loss D:  2.9162
Epoch 7, Loss G:  1.0212, Loss D:  1.9030
Epoch 8, Loss G:  1.0194, Loss D:  0.9773
Epoch 9, Loss G:  0.9851, Loss D:  0.3535
Epoch 10, Loss G:  0.9674, Loss D:  0.0930
Epoch 11, Loss G:  0.9709, Loss D: -0.0399
Epoch 12, Loss G:  0.9091, Loss D:  0.0079
Epoch 13, Loss G:  0.9035, Loss D:  0.0732
Epoch 14, Loss G:  0.8422, Loss D:  0.0973
Epoch 15, Loss G:  0.9111, Loss D:  0.0771
Epoch 16, Loss G:  0.8757, Loss D:  0.0040
Epoch 17, Loss G:  0.8699, Loss D:  0.0158
Epoch 18, Loss G:  0.8876, Loss D:  0.0727
Epoch 19, Loss G:  0.9004, Loss D:  0.0446
Epoch 20, Loss G:  0.8745, Loss D:  0.1027
Epoch 21, Loss G:  0.8595, Loss D:  0.1220
Epoch 22, Loss G:  0.8249, Loss D:  0.0591
Epoch 23, Loss G:  0.8266, Loss D:  0.1097
Epoch 24, Loss G:  0

In [14]:
data = train.synthesise_data(100)
data

,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year
0,9621,-382,54167,Suburu,TL,2012
1,2258,8160,55810,Saab,Ultima,1999
2,8720,6667,64109,Chevrolet,Wrangler,2007
3,873,8742,35488,Dodge,F150,2002
4,10307,86,41173,Volkswagen,A5,1999
...,...,...,...,...,...,...
95,17533,1281,59445,Nissan,Jetta,2006
96,8635,16960,45451,Honda,Escape,2013
97,10294,11514,59724,Dodge,Ultima,1998
98,14833,13136,62203,Jeep,Corolla,2009


In [15]:
data2 = train.synthesise_data(50, condition_column='auto_make', condition_value='Jeep')
data2

,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year
0,1342,13236,80547,Honda,Grand Cherokee,1999
1,18324,406,36460,Nissan,Wrangler,2012
2,14009,14941,52996,Suburu,TL,2006
3,8232,5240,65834,Toyota,92x,2000
4,11983,10003,50959,Jeep,Impreza,1995
5,1945,8699,7084,Chevrolet,A5,2003
6,1894,9421,60620,Dodge,Escape,2003
7,1429,1311,5978,Honda,Corolla,2001
8,19467,2484,3439,Toyota,Maxima,1995
9,7262,12534,5085,Nissan,Forrestor,2000
